# Model 37


|Markets   |    X  |   Y   |  K  |   W     |     CONS  |
|----------|-------|-------|-----|---------|-----------|
|PX        |   100 |       |     | -100    | |
|PY        |       |  100  |     | -100    | |
|PW        |       |       |     |  200    |     -200|
|PL        |   -40 |  -60  | -60 |         |      160|
|PK        |  -120 |  -80  |  60 |         |      140|
|SUB       |    60 |   40  |     |         |     -100|


In [1]:
using MPSGE_MP

In [9]:
M37 = MPSGEModel()


@parameters(M37, begin
    RHO, .4, (description = "Time preference parameter,",)
    DELTA, .3, (description = "Depreciation rate,",)
    TAU, -(1-.3)/(1+.4), (description = "Effective capital use tax (it is a subsidy, its < 0),",)
    KTAX, 0, (description = "Tax on new capital production",)
    NEWCAP, 0, (description = "New capital stock after counterfactual (= 1 initially)",)
end)


@sectors(M37, begin
	X,	(description = "Activity level for sector X",)
	Y,	(description = "Activity level for sector Y",)
	W,	(description = "Activity level for sector W (Hicksian welfare index)",)
	K,	(description = "Capital stock index",)
end)

@commodities(M37, begin
	PX,	(description = "Price index for commodity X",)
	PY,	(description = "Price index for commodity Y",)
	PL,	(description = "Price index for primary factor L (net of tax)",)
	PK,	(description = "Price index for primary factor K",)
	PW,	(description = "Price index for welfare (expenditure function)",)
end)

@consumers(M37, begin
	CONS,	(description = "Income level for consumer CONS",)
end)

@auxiliaries(M37, begin
	KFORWRD,	(description = "Capital stock from previous period",)
end)


@production(M37, X, [s=1, t=0], begin
    @output(PX, 100, t)
    @input(PL, 40, s)
    @input(PK, 120, s, taxes = [Tax(CONS,TAU)], reference_price = .5)
end)

@production(M37, Y, [s=1, t=0], begin
    @output(PY, 100, t)
    @input(PL, 60, s)
    @input(PK, 80, s, taxes = [Tax(CONS,TAU)], reference_price = .5)
end)

@production(M37, K, [s=0,t=0], begin
    @output(PK, 1, t, taxes = [Tax(CONS, KTAX)])
    @input(PL, 1, s)
end)

@production(M37, W, [s=1,t=0], begin
    @output(PW, 200, t)
    @input(PX, 100, s)
    @input(PY, 100, s)
end)

@demand(M37, CONS, begin
    @final_demand(PW, 200)
end,begin
    @endowment(PL, 160)
    @endowment(PK, KFORWRD)
end)

@aux_constraint(M37, KFORWRD,
    KFORWRD - K*(1-DELTA)/DELTA
)

ScalarAuxConstraint(KFORWRD, KFORWRD - ((-K*DELTA + K) / DELTA))

# Benchmark

In [10]:
set_start_value(K, 60)
set_start_value(KFORWRD, 140)
fix(PW, 1)

solve!(M37; cumulative_iteration_limit=0)
generate_report(M37)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_990D.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 10

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,1.0,0.0
2,X,1.0,0.0
3,W,1.0,0.0
4,PL,1.0,0.0
5,PW,1.0,0.0
6,PK,1.0,0.0
7,PY,1.0,0.0
8,PX,1.0,0.0
9,CONS,200.0,0.0


# Counterfactual

In [16]:
set_value!(RHO, .6)
set_value!( TAU, -(1-value(DELTA))/(1+value(RHO)))

solve!(M37)
generate_report(M37)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_3162.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 10

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.7743e+01             0.0e+00 (market_clearance[PK)
    1     1     0    10 4.9465e-01  1.0e+00    0.0e+00 (income_balance[CONS)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 4.9465e-01           I 0.0e+00 4.9e-01 (income_balance[)
    1     1     3     3 1.6053e-07  1.0e+00 SO 0.0e+00 1.6e-07 (income_balance[)

Major Iterations. . . . 1
Minor Iterations. . . . 1
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 3
Gradient Evaluations. . 3
Basis Time. . . . . .

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,0.995457,-4.26326e-14
2,X,0.972281,1.42109e-14
3,W,0.983801,0.0
4,PL,0.942809,-5.68434e-14
5,PW,1.0,-1.60526e-7
6,PK,0.942809,2.84217e-14
7,PY,0.988291,-1.42109e-14
8,PX,1.01185,-5.68434e-14
9,CONS,196.76,-1.60526e-7


# Counterfactual

In [17]:
set_value!(RHO, .4)
set_value!( TAU, -(1-value(DELTA))/(1+value(RHO)))

set_value!(KTAX,.2)

solve!(M37)
generate_report(M37)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_BCEA.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 10

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.2361e+01             0.0e+00 (income_balance[CONS)
    1     1     0    10 2.1756e+00  1.0e+00    0.0e+00 (income_balance[CONS)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 2.1756e+00           I 0.0e+00 1.7e+00 (income_balance[)
    1     1     3     3 6.9930e-03  1.0e+00 SO 0.0e+00 7.0e-03 (income_balance[)
    2     1     4     4 1.0173e-13  1.0e+00 SO 0.0e+00 5.7e-14 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . 

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,0.988768,-2.13163e-14
2,X,0.94561,-1.42109e-14
3,W,0.966948,2.84217e-14
4,PL,0.894427,0.0
5,PW,1.0,-1.13687e-13
6,PK,1.11803,-2.84217e-14
7,PY,0.977933,2.84217e-14
8,PX,1.02257,5.68434e-14
9,CONS,193.39,-2.84217e-14
